In [1]:
import arcpy,os
import pandas as pd
import json 


In [2]:
def handle_shp_gdb_path(sourcepath) :
    """handles different path formats for shapefiles and geodatabases

    If the path is a shapefie it returns just the path. If the path is a geodatabase it returns also the name of the geo-database.

    Args:
        sourcepath (Path): path relative to the project root

    Returns:
        Path: path of the shapefile or geodatabase
    """
    path = os.path.normpath(sourcepath)
    splitted = path.split(os.sep)

    if splitted[-1].endswith(".shp"):
        # if the path is a shapefile it returns just the path
        return sourcepath
    else:
        # if the path is a geodatabase it returns also the name of the geo-database
        for i,ii in zip(splitted,range(len(splitted))):
            if i.endswith(".gdb"):
                return os.path.join(*splitted[ii:])



In [9]:
def get_layer_info(imxd):
    """get info about the layers in the mxd file

    Args:
        mxd_path : mxd object

    Returns:
        dict: dictionary with the layer names and the paths to the shapefiles or geodatabases
    """

    lyrs = arcpy.mapping.ListLayers(imxd)

    layers={}
    layers["feature_layers"]={}
    layers["errors"]={}
    layers["broken_layers"]=[]
    layers["basemap_layers"]=[]
    layers["group_layers"]=[]
    for lyr in lyrs:
        if lyr.isBroken:
            layers["broken_layers"].append(lyr.name)
        else:
            # hardcoded to force skip this layers
            if not any(substring in lyr.name for substring in ["World Imagery","Imagery","Metadata","Citations","Default"]):
                if lyr.isFeatureLayer:
                    try:
                        layers["feature_layers"][lyr.name]=handle_shp_gdb_path(lyr.dataSource)
                    except Exception as e:
                        layers["errors"][lyr.name]=(e)
                elif lyr.isBasemapLayer:
                    layers["basemap_layers"].append(lyr.name)
                elif lyr.isGroupLayer:
                    layers["group_layers"].append(lyr.name)
                else:
                    print(lyr.name)
                    print("isFeatureLayer: ",lyr.isFeatureLayer)
                    print("isBasemapLayer: ",lyr.isBasemapLayer)
                    print("isGroupLayer: ",lyr.isGroupLayer)
                    print("isNetworkAnalystLayer: ",lyr.isNetworkAnalystLayer)
                    print("isRasterLayer: ",lyr.isRasterLayer)
                    print("isServiceLayer: ",lyr.isServiceLayer)
                    print("isRasterizingLayer: ",lyr.isRasterizingLayer)
                    print(lyr.supports)
                    raise NotImplementedError("Layer type not implemented")
    return layers

In [15]:
folderPath = r"C:\Users\GIORGIO.CAIZZI\AECOM\60678625ER Marghera - AdR VP e NP - 09_Temp\00 - GIS\02 - Tavole Modello Concettuale\Nuovo petrolchimico"

tavole={}

for filename in os.listdir(folderPath):
    fullpath = os.path.join(folderPath, filename)
    if os.path.isfile(fullpath):
        basename, extension = os.path.splitext(fullpath)
        if extension.lower() == ".mxd":
            filename=filename.decode("latin8")
            print(filename)
            imxd=arcpy.mapping.MapDocument(fullpath)
            tavole[filename]=get_layer_info(imxd)
            del imxd


A.01 - Planimetria confini proprietà e recettori off-site.mxd
A.02 - Aree interessate da interventi di ripristino.mxd
A.03 - Planimetria ubicazione indagini.mxd
A.09 - Planimetria ubicazione piezometri e concentrazione acque sotterranee.mxd
A.10 - Ubicazione presenza fonti primarie di contaminazione e MMR.mxd
A.12.01 - Sorgenti di potenziale contaminazione SS CD Scenario 1.mxd
A.12.02 - Sorgenti di potenziale contaminazione SS IN Scenario 1.mxd
A.12.03 - Sorgenti di potenziale contaminazione SP IN Scenario 1.mxd
A.12.04 - Sorgenti di potenziale contaminazione GW Scenario 1.mxd
A.12.05 - Sorgenti di potenziale contaminazione SS CD Scenario 2.mxd
A.12.06 - Sorgenti di potenziale contaminazione SS IN Scenario 2.mxd
A.12.07 - Sorgenti di potenziale contaminazione SP CD Scenario2.mxd
A.12.08 - Sorgenti di potenziale contaminazione SP IN Scenario 2.mxd
A.12.09 - Sorgenti di potenziale contaminazione GW Scenario 2.mxd
A.13.01 - Planimetria ubicazione eccedenze delle CSC per i contaminanti nel

In [16]:
tavole

{u'A.01 - Planimetria confini propriet\xe0 e recettori off-site.mxd': {'basemap_layers': [u'Basemap'],
  'broken_layers': [],
  'errors': {},
  'feature_layers': {u'Lotti': u'NP_ADR_2021.gdb\\lotti',
   u'recettori_rev2': u'NP_ADR_2021.gdb\\recettori_rev2'},
  'group_layers': []},
 u'A.02 - Aree interessate da interventi di ripristino.mxd': {'basemap_layers': [],
  'broken_layers': [],
  'errors': {},
  'feature_layers': {u'Aree soggette ad attivit\xe0 di scotico': u'NP_ADR_2021.gdb\\scotico',
   u'Carta Tecnica Regionale': u'NP_ADR_2021.gdb\\ctr',
   u'Lotti': u'NP_ADR_2021.gdb\\lotti',
   u'MMR': u'NP_ADR_2021.gdb\\NP_esiti_rev2',
   u'Poligoni di thiessen': u'NP_ADR_2021.gdb\\thiessen_SS_CD_rev2',
   u'Poligoni oggetto di attivit\xe0 di bonifica': u'NP_ADR_2021.gdb\\misp',
   u'Poligono oggetto di ripristino ambientale (MT-S25-S2)': u'NP_ADR_2021.gdb\\thiessen_SS_CD_rev2',
   u'all_polys': u'NP_ADR_2021.gdb\\Aree_MISP',
   u'terreno': u'NP_ADR_2021.gdb\\NP_esiti_rev2',
   u'thiessen

In [17]:
# Encode the strings to UTF-8
encoded_tavole = {k: str(v).decode("utf-8") if isinstance(v, str) else v for k, v in tavole.iteritems()}


In [18]:
encoded_tavole

{u'A.01 - Planimetria confini propriet\xe0 e recettori off-site.mxd': {'basemap_layers': [u'Basemap'],
  'broken_layers': [],
  'errors': {},
  'feature_layers': {u'Lotti': u'NP_ADR_2021.gdb\\lotti',
   u'recettori_rev2': u'NP_ADR_2021.gdb\\recettori_rev2'},
  'group_layers': []},
 u'A.02 - Aree interessate da interventi di ripristino.mxd': {'basemap_layers': [],
  'broken_layers': [],
  'errors': {},
  'feature_layers': {u'Aree soggette ad attivit\xe0 di scotico': u'NP_ADR_2021.gdb\\scotico',
   u'Carta Tecnica Regionale': u'NP_ADR_2021.gdb\\ctr',
   u'Lotti': u'NP_ADR_2021.gdb\\lotti',
   u'MMR': u'NP_ADR_2021.gdb\\NP_esiti_rev2',
   u'Poligoni di thiessen': u'NP_ADR_2021.gdb\\thiessen_SS_CD_rev2',
   u'Poligoni oggetto di attivit\xe0 di bonifica': u'NP_ADR_2021.gdb\\misp',
   u'Poligono oggetto di ripristino ambientale (MT-S25-S2)': u'NP_ADR_2021.gdb\\thiessen_SS_CD_rev2',
   u'all_polys': u'NP_ADR_2021.gdb\\Aree_MISP',
   u'terreno': u'NP_ADR_2021.gdb\\NP_esiti_rev2',
   u'thiessen

In [19]:
with open('tavole.json', 'w+') as fp:
    json.dump(encoded_tavole, fp, indent=4)